In [1]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=[
        "#4B9AC7",
        "#4BE8E0",
        "#9DD4F3",
        "#97FBF6",
        "#2A7FAF",
        "#23B1AB",
        "#0E3449",
        "#015955",
    ]
)
pio.templates.default = "jedha"

In [2]:
df_delay = pd.read_excel("src/get_around_delay_analysis.xlsx")

In [3]:
df_delay.to_csv("get_around_delay_analysis.csv", index=False)

In [3]:
df_delay.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


In [4]:
df_delay.describe(include="all")

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000


In [5]:
df_delay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB


In [6]:
px.box(df_delay, x="delay_at_checkout_in_minutes")

50 days late seems to be an error, removing outliers

In [7]:
Q1 = df_delay["delay_at_checkout_in_minutes"].quantile(0.25)
Q3 = df_delay["delay_at_checkout_in_minutes"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound =  Q3 + 1.5 * IQR
print( lower_bound, upper_bound)

-190.5 221.5


In [8]:
df_delay_clean = df_delay.copy()
df_delay_clean["delay_at_checkout_in_minutes"] = df_delay_clean["delay_at_checkout_in_minutes"].apply(lambda x : x if x >= lower_bound and x <= upper_bound else float("nan")) #[((df_delay["delay_at_checkout_in_minutes"] >= Q1 - 1.5 * IQR) & (df_delay["delay_at_checkout_in_minutes"] <= Q3 + 1.5 * IQR)) | (df_delay["delay_at_checkout_in_minutes"].isna())].copy()

In [9]:
# geting previous checkout delay if available
df_delay_clean["previous_rental_delay_at_checkout_in_minutes"] = df_delay_clean["previous_ended_rental_id"].apply(lambda x : df_delay_clean[df_delay_clean["rental_id"] == x]["delay_at_checkout_in_minutes"].iloc[0] if len(df_delay_clean[df_delay_clean["rental_id"] == x]) > 0 else None)
# calculating if car was late
df_delay_clean["time_delta_since_car_last_checkout_in_minutes"] = df_delay_clean["previous_rental_delay_at_checkout_in_minutes"]  - df_delay_clean["time_delta_with_previous_rental_in_minutes"]
df_delay_clean["car_is_late_for_next_checkin"] = df_delay_clean["time_delta_since_car_last_checkout_in_minutes"].apply(lambda x : x > 0) 

In [10]:
df_delay_clean_with_delay = df_delay_clean.loc[df_delay_clean["delay_at_checkout_in_minutes"].notna(),:].copy() # Removing rows without checkout delay (also remove all canceled rentals)

In [11]:
px.box(df_delay_clean_with_delay, x="delay_at_checkout_in_minutes")

In [12]:
px.histogram(df_delay_clean_with_delay, x="delay_at_checkout_in_minutes",color="checkin_type",barmode="group", histnorm="percent")

rentals with connect checkin tends to check_out earlier.

In [13]:
px.box(df_delay_clean_with_delay, x="time_delta_with_previous_rental_in_minutes")

Half of rentals with a previous rental have less than 3 hour delay

In [14]:
df_delay_clean["car_is_late_for_next_checkin"]

0        False
1        False
2        False
3        False
4        False
         ...  
21305    False
21306    False
21307    False
21308    False
21309    False
Name: car_is_late_for_next_checkin, Length: 21310, dtype: bool

In [15]:
fig = px.histogram(df_delay_clean[(df_delay_clean["time_delta_since_car_last_checkout_in_minutes"].notna())],  x="state", color="car_is_late_for_next_checkin", histnorm="percent",barmode="group")
fig.show()

When the car is late 4% more rentals are canceled

In [16]:

df_no_delay = df_delay_clean[(df_delay_clean["time_delta_with_previous_rental_in_minutes"]== 0) & (df_delay_clean["time_delta_since_car_last_checkout_in_minutes"].notna())]
df_last_car_checkout_not_NaN = df_delay_clean[df_delay_clean["time_delta_since_car_last_checkout_in_minutes"].notna()]
df_last_car_checkout_late = df_delay_clean[df_delay_clean["car_is_late_for_next_checkin"] == True]
fig = px.histogram(df_delay_clean, x="time_delta_since_car_last_checkout_in_minutes", histnorm="percent")
fig.show()
fig = px.histogram( df_no_delay, x="time_delta_since_car_last_checkout_in_minutes", histnorm="percent")
fig.show()
fig = px.histogram(df_last_car_checkout_not_NaN, x="car_is_late_for_next_checkin", histnorm="percent")
fig.show()
fig = px.histogram(df_no_delay, x="car_is_late_for_next_checkin", histnorm="percent")
fig.show()

fig = px.histogram(df_last_car_checkout_late, x="time_delta_with_previous_rental_in_minutes",y="time_delta_since_car_last_checkout_in_minutes", histfunc="avg", nbins=30)
fig.update_layout(
    xaxis=dict(
        tickvals=[0, 30, 60, 90, 120, 150, 180],
        ticktext=['0', '30', '60', '90', '120', "150", "180"],
        title='time delta with previous rental in minutes'
    )
)
fig.show()

fig = px.histogram(df_last_car_checkout_not_NaN, x="car_is_late_for_next_checkin", color="time_delta_with_previous_rental_in_minutes", barmode="group", histnorm="percent")

fig.show()

46% of rentals that directly follow another rental without any interval period are late. In contrast, only 23% of rentals are late when there is a 30-minute gap between them, although the average delay is longer.

In [17]:

fig = px.violin(df_delay_clean,x="delay_at_checkout_in_minutes", title="delay at checkout distribution")
fig.update_layout(
    xaxis=dict(
        title='delay at checkout in minutes'
    )
)

In [18]:
canceled_rentals_given_late = df_delay_clean[(df_delay_clean["time_delta_since_car_last_checkout_in_minutes"].notna()) & (df_delay_clean["car_is_late_for_next_checkin"])]["state"].apply(lambda x : True if x == "canceled" else False).mean() * 100
canceled_rentals_given_not_late = df_delay_clean[(df_delay_clean["time_delta_since_car_last_checkout_in_minutes"].notna()) & (df_delay_clean["car_is_late_for_next_checkin"] == False)]["state"].apply(lambda x : True if x == "canceled" else False).mean() * 100
fig = go.Figure(data=[
    go.Bar(
        x=["late for the checkin","On time for the checkin"],  # Convert to string for better display
        y=[canceled_rentals_given_late, canceled_rentals_given_not_late]
        
    )
])
# Update layout
fig.update_layout(
    title='Percentage of Cancellations by Car is Late for Next Check-in',
    yaxis_title='Cancellation Percentage'
)
fig.show()

In [19]:
df_delay_clean[(df_delay_clean["time_delta_since_car_last_checkout_in_minutes"] >= 70) & (df_delay_clean["time_delta_since_car_last_checkout_in_minutes"] < 80)]

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_rental_delay_at_checkout_in_minutes,time_delta_since_car_last_checkout_in_minutes,car_is_late_for_next_checkin
3197,554062,394024,mobile,ended,-48.0,562173.0,0.0,73.0,73.0,True
12754,564913,407315,mobile,ended,-172.0,565399.0,0.0,70.0,70.0,True
13330,557773,402858,mobile,ended,-68.0,511719.0,0.0,71.0,71.0,True
13335,557990,312572,mobile,ended,-89.0,558495.0,0.0,73.0,73.0,True
13862,552207,360551,connect,ended,68.0,550621.0,30.0,108.0,78.0,True
19340,568224,381499,connect,ended,-93.0,562174.0,0.0,73.0,73.0,True


In [20]:
# Create the initial bar plot
fig = go.Figure()
fig.add_trace(go.Bar(
        x=["on time"],
        y=[canceled_rentals_given_not_late]
    ))
fig.add_trace(go.Bar(
        x=["late (any delay)"],
        y=[canceled_rentals_given_late]
    ))
step = 50
# Add bars using a loop
for i in range(0,200,step):
    canceled_rentals_given_late_from_i = df_delay_clean[(df_delay_clean["time_delta_since_car_last_checkout_in_minutes"] >= i) & (df_delay_clean["time_delta_since_car_last_checkout_in_minutes"] < i + step)]["state"].apply(lambda x : True if x == "canceled" else False).mean() * 100
    fig.add_trace(go.Bar(
        x=[f"{i}-{i+step}"],
        y=[canceled_rentals_given_late_from_i]
    ))
fig.update_layout(
    title="Analyzing the impact of check-in delays on cancellation rates",
    xaxis_title='Delay in minutes',
    yaxis_title='Cancellation Percentage',
    showlegend = False
)
fig.show()

In [21]:
pd.DataFrame({"threshold" : [1]})

,threshold
0,1


In [22]:

df_len = len(df_delay_clean)
df_delay_len = len(df_delay_clean_with_delay)
# connect
df_connect = df_delay_clean.loc[df_delay_clean["checkin_type"] == "connect"]
df_delay_connect = df_delay_clean_with_delay.loc[df_delay_clean["checkin_type"] == "connect"]
df_connect_len = len(df_connect)
df_delay_connect_len = len(df_delay_connect)
# mobile
df_mobile = df_delay_clean.loc[df_delay_clean["checkin_type"] == "mobile"]
df_delay_mobile = df_delay_clean_with_delay.loc[df_delay_clean["checkin_type"] == "mobile"]
df_mobile_len = len(df_mobile)
df_delay_mobile_len = len(df_delay_mobile)

In [23]:
df_thresholds = pd.DataFrame({"threshold" : [i for i in range(0,241,30) ]})
df_thresholds["share_affected_connect"] = df_thresholds["threshold"].apply(lambda x : len(df_connect[(df_connect["time_delta_with_previous_rental_in_minutes"] < x)])) # Consider NaN values as durations exceeding 12 hours.
df_thresholds["share_affected_percentage_connect"] = df_thresholds["share_affected_connect"].apply(lambda x : x / df_connect_len * 100)
df_thresholds["share_affected_percentage_all_connect"] = df_thresholds["share_affected_connect"].apply(lambda x : x / df_len * 100)
df_thresholds["late_checkout_connect"] = df_thresholds["threshold"].apply(lambda x : len(df_delay_connect[(df_delay_connect["delay_at_checkout_in_minutes"] > x)]))
df_thresholds["late_checkout_probability_connect"] = df_thresholds["late_checkout_connect"].apply(lambda x : x / df_delay_connect_len * 100)
df_thresholds["late_checkout_probability_all_connect"] = df_thresholds["late_checkout_connect"].apply(lambda x : x / df_delay_len * 100)

df_thresholds["share_affected_mobile"] = df_thresholds["threshold"].apply(lambda x : len(df_mobile[(df_mobile["time_delta_with_previous_rental_in_minutes"] < x)])) # Consider NaN values as durations exceeding 12 hours.
df_thresholds["share_affected_percentage_mobile"] = df_thresholds["share_affected_mobile"].apply(lambda x : x / df_mobile_len * 100)
df_thresholds["share_affected_percentage_all_mobile"] = df_thresholds["share_affected_mobile"].apply(lambda x : x / df_len * 100)
df_thresholds["late_checkout_mobile"] = df_thresholds["threshold"].apply(lambda x : len(df_delay_mobile[(df_delay_mobile["delay_at_checkout_in_minutes"] > x)]))
df_thresholds["late_checkout_probability_mobile"] = df_thresholds["late_checkout_mobile"].apply(lambda x : x / df_delay_mobile_len * 100)
df_thresholds["late_checkout_probability_all_mobile"] = df_thresholds["late_checkout_mobile"].apply(lambda x : x / df_delay_len * 100)

In [24]:
df_thresholds

,threshold,share_affected_connect,share_affected_percentage_connect,share_affected_percentage_all_connect,late_checkout_connect,late_checkout_probability_connect,late_checkout_probability_all_connect,share_affected_mobile,share_affected_percentage_mobile,share_affected_percentage_all_mobile,late_checkout_mobile,late_checkout_probability_mobile,late_checkout_probability_all_mobile
0,0,0,0.000000,0.000000,1358,45.924924,9.758551,0,0.000000,0.000000,6608,60.297472,47.484909
1,30,131,3.041560,0.614735,777,26.276632,5.583501,148,0.870435,0.694510,3863,35.249567,27.759414
2,60,181,4.202461,0.849366,452,15.285763,3.248060,220,1.293889,1.032379,2496,22.775801,17.936189
3,90,260,6.036684,1.220084,278,9.401420,1.997700,324,1.905546,1.520413,1541,14.061502,11.073584
4,120,295,6.849315,1.384327,160,5.410889,1.149756,371,2.181968,1.740967,955,8.714299,6.862604
5,150,345,8.010216,1.618958,92,3.111261,0.661110,458,2.693642,2.149226,539,4.918332,3.873239
6,180,372,8.637102,1.745659,46,1.555631,0.330555,498,2.928895,2.336931,272,2.481978,1.954585
7,210,409,9.496169,1.919287,7,0.236726,0.050302,544,3.199435,2.552792,57,0.520120,0.409600
8,240,430,9.983747,2.017832,0,0.000000,0.000000,571,3.358231,2.679493,0,0.000000,0.000000


In [30]:
threshold_connect = 30
threshold_mobile = 90

In [42]:
connect_share_affected = df_thresholds[df_thresholds["threshold"] == threshold_connect]["share_affected_percentage_all_connect"].iloc[0]
mobile_share_affected = df_thresholds[df_thresholds["threshold"] == threshold_mobile]["share_affected_percentage_all_mobile"].iloc[0]
unafected_share = 100 - connect_share_affected - mobile_share_affected 

connect_late_probability = df_thresholds[df_thresholds["threshold"] == threshold_connect]["late_checkout_probability_all_connect"].iloc[0]
mobile_late_probability = df_thresholds[df_thresholds["threshold"] == threshold_mobile]["late_checkout_probability_all_mobile"].iloc[0]
on_time = 100 - connect_late_probability - mobile_late_probability 

# Create subplots: 1 row, 2 columns for pie charts
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]])



fig.add_trace(go.Pie(labels=["connect","mobile","unaffected"],values=[connect_share_affected, mobile_share_affected, unafected_share], title={'text':'Share affected', 'position': 'bottom center'}, hole=0.6 ),row=1, col=1)
fig.add_trace(go.Pie(labels=["connect","mobile","on time"],values=[connect_late_probability, mobile_late_probability, on_time], title={'text':'Probability to be late for checkin at minimum interval', 'position': 'bottom center'}, hole=0.6 ),row=1, col=2)

fig.add_trace(go.Scatter(x=df_thresholds['threshold'],y=df_thresholds['share_affected_percentage_connect'], name="Share affected (connect)", visible=False ))
fig.add_trace(go.Scatter(x=df_thresholds['threshold'],y=df_thresholds['late_checkout_probability_connect'], name="Probability to be late at minimum interval (connect)", visible=False))

fig.add_trace(go.Scatter(x=df_thresholds['threshold'],y=df_thresholds["share_affected_percentage_mobile"], name="Share affected (mobile)", visible=False ))
fig.add_trace(go.Scatter(x=df_thresholds['threshold'],y=df_thresholds["late_checkout_probability_mobile"], name="Probability to be late at minimum interval (mobile)", visible=False))


fig.add_shape(type='line',
              x0=threshold_connect, x1=threshold_connect,
              y0=0, y1=1,
              yref='paper', line=dict(color='red', width=2),
              visible=False)

fig.add_shape(type='line',
              x0=threshold_mobile, x1=threshold_mobile,
              y0=0, y1=1,
              yref='paper', line=dict(color='red', width=2),
              visible=False)



# Create the layout
layout = go.Layout(
    title="Affected rentals",
    xaxis = {"visible" : False},
    yaxis = {"visible" : False},
    xaxis_title='Threshold in minutes',
    yaxis_title='Affected renatals Percentage',
    updatemenus=[
        {
            
            "buttons": [
                {
                    "label": "Affected rentals",
                    "method": "update",
                    "args": [
                        {"visible": [True, True, False, False, False, False]}, 
                        {
                            "title" : "Affected rentals", 
                            "shapes[0].visible": False, 
                            "shapes[1].visible": False, 
                            "xaxis" : {"visible" : False}, 
                            "yaxis" : {"visible" : False} 
                        }
                    ]
                },
                {
                    "label": "Affected rentals (connect)",
                    "method": "update",
                    "args": [
                        {"visible": [False, False, True, True, False, False]}, 
                        {
                            "title" : "Affected rentals by threshold (connect)", 
                            "shapes[0].visible": True, 
                            "shapes[1].visible": False, 
                            "xaxis" : {"visible" : True, "title": {"text": "Threshold in minutes"}}, 
                            "yaxis" : {"visible" : True, "title": {"text": "Affected rentals Percentage"}}
                        }
                    ]
                },
                {
                    "label": "Affected rentals (mobile)",
                    "method": "update",
                    "args": [
                        {"visible": [False, False, False, False, True, True]}, 
                        {
                            "title" : "Affected rentals by threshold (mobile)", 
                            "shapes[0].visible": False, 
                            "shapes[1].visible": True, 
                            "xaxis" : {"visible" : True, "title": {"text": "Threshold in minutes"}}, 
                            "yaxis" : {"visible" : True, "title": {"text": "Affected rentals Percentage"}}
                        }
                    ]
                }
            ],
            "direction": "down",
            "showactive": True,
        }
    ]
)
# Update layout
fig.update_layout(layout)

fig.show()